# Utilisation du machine learning pour le décodage du cerveau

- extraction de features?
- selection algo
    * LDA, SVM, Arbres...
    * En fonction de la taille et de la forme des données.
- selection du design de l'expérience
    * Intra-sujets (BCI).
    * inter-sujets recherche plus fondamentale.
    * K-Fold, stratified K-fold, Leave-one-out, leave-p-out, stratified leave-p-out.
    * métrique utilisée pour évaluer le modèle.
- evaluation du resultat
    * Visualisations (matrices de confusion, topomaps)
    * Test de permutations
    * corrections statistiques
- Bonus
    * Optimisation hyper-paramètres
    * Multi-features
    * Deep learning -> se passer de la l'extraction de caractéristiques

## Définition du problème

Avant de se lancer directement dans les données, il faut déjà savoir quelles caractéristiques du signal on compte utiliser et choisir un modèle adapté.

Tout les algorithmes d'apprentissage automatique ne sont pas égaux face à différents jeux de données. Par exemple, les forêts et arbres de classification sont plus efficaces sur des données avec beaucoup de caractéristiques que les SVM qui souffriront bien plus du fléau de la dimensionalité.

Il faut aussi se demander si la quantité de données est suffisante pour l'hypothèse que l'on veut vérifier. Même si on peut parfois utiliser des tests statistiques pour valider un résultat obtenu avec peu de données. Comme en neuroscience, les données dont souvent coûteuses et donc souvent peu nombreuses il n'est pas rare de devoir avoir recours à de l'augmentation de données pour y pallier.

Dans le cas présent, nous allons explorer différents designs pour essayer de résoudre différents problèmes sur un même jeu de données.

Pour se faire, on va utiliser la librairie <a href="">sklearn</a> qui va nous permettre d'utiliser certains algorithmes d'apprentissagge machine assez simplement.

### Installation des librairies nécessaires:

In [ ]:
!pip install sklearn numpy matplotlib

### Imports

In [11]:
import os.path as op
import numpy as np
from sklearn.model_selection import KFold, LeaveOneGroupOut, LeavePGroupsOut, StratifiedKFold
from sklearn.svm import SVC as SVM
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RF
from matplotlib import pyplot as plt

%matplotlib inline

## Problème 1: Classification gauche-droite

Premièrement nous allons nous intéresser à effectuer une classification de nos données qui nous permettra de détecter si le sujet à entendu un stimulus à gauche ou bien à droite. Dans une approche guidée par les données (data driven), nous n'allons pas émetre d'hypothèse au préalable sur les résultats. 

On va, pour chaque capteur, essayer de faire la différence entre stimulus gauche et stimulus droit, en utilisant les valeurs de PSD calculées dans l'AP3.

### Chargement des données

Les classifications sont des méthodes d'apprentissage machine dites supervisées et nécessitent l'utilisation de deux matrices:
- une matrice data souvent appelée X, qui contient les données que l'on va utiliser pour l'apprentissage et le test de l'algorithme d'apprentissage machine.
- Une matrice "labels" ou "targets" souvent appelée y, qui contient l'information que l'on souhaite prédire encodée en chiffres.

Dans notre exemple, comme nous souhaitons classifier entre stimulus gauche ou droite, on va arbitrairement associer 0 ou 1 à chacune de nos classes. Par exemple: 0 <=> gauche ; 1 <=> droite.

In [84]:
conditions = ['visualleft', 'visualright']
# data_path = mne.datasets.sample.data_path()
data_path = "/home/arthur/Downloads/"
data = []
targets = []
for i, condition in enumerate(conditions):
#     temp_data = np.load(op.join(data_path, 'MEG', 'sample', f'{condition}_psds_bands'))
    temp_data = np.load(op.join(data_path, f'{condition}_psds_bands.npy'))
    targets += [i]*temp_data.shape[1]
    data.append(temp_data)

X = np.concatenate(data, axis=1)
y = np.array(targets)

### Choix de l'algorithme

Pour commencer, nous allons utiliser K Nearest Neighbors (KNN). Un algorithms simple, rapide en exécution qui possède un seul hyper-paramètre.

Pour le moment, on va évaluer les performances de l'algorithme sans se soucier d'optimiser les hyper-paramètres, mais on reviendra sur ce concept plus tard.

In [17]:
clf = KNN()

### Choix de la méthode de validation

Il nous faut maintenant choisir une méthode qui nous permettra de mieux évaluer les performances de notre algorithme. Le choix de cette méthode est important car si la méthode de validation est mal choisie, elle pourrait impacter nos résultats. 

Cette méthode va dicter la façon dont on va séparer nos données afin d'entraîner notre algorithme sur une partie et de tester sur une autre. Elle est nécessaire car sans elle, nous ne pourrions pas controller le sur-apprentissage (overfitting).

La méthode la plus simple qui existe est de séparer nos données en deux partie: entraînement (train) et validation ou test ce que l'on appelle la "Holdout method":

Coupons arbitrairemement les données en 2:

In [92]:
X_train = X[:72]
y_train = y[:72]

X_test = X[72:]
y_test = y[72:]

Immédiatement, si on affiche les labels de nos données on se rend compte d'un problème: notre sous-ensemble d'etraaînement ne contient que des exemples de stimuli à gauche et sera donc incapable d'apprendre comment différentier droite de gauche!

In [87]:
print(y_train)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Il nous faut donc prendre au hasard des exemples du jeu de données initial ou encore le mélanger avant de tirer les exemples.

Pour ce faire, on créé un index des données sur lequel on va effectuer le mélange afin de pouvoir effectuer exactement le même mélamge sur les labels. Sans quoi on perdrait la correspondance entre exemple et sa catégorie:

In [100]:
# Création de l'index:
index = np.arange(X.shape[1])

# mélange de l'index:
np.random.shuffle(index)

# On applique le mélange aux données et aux labels:
shuffled_X = X[index]
shuffled_y = y[index]

# On effectue notre séparation (split) à nouveau:
X_train = shuffled_X[:72]
y_train = shuffled_y[:72]

X_test = shuffled_X[72:]
y_test = shuffled_y[72:]

# Affichons à nouveau nos labels d'entraînement pour vérifier:
print(y_train)

[1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1
 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 1 0 1 1 1 0]


En pratique, sklearn implémente des fonctions qui vont permettre de faciliter ce travai:

In [111]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.swapaxes(0,1), y, test_size=72)
X_train = X_train.swapaxes(0,1)
X_test = X_test.swapaxes(0,1)

print(y_train)

[1 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 1 1 0 0 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0]


Note: Ici on est obligé d'utiliser "swapaxes" car il faut que la première dimension de nos données soit la dimension qui liste les exemples. Or actuellement, la première dimentsion est la dimension des électrodes.

Note2: Étant donné que le mélange des données est aléatoire, il est toujours possible par malchance d'avoir un split qui favorise une classe dans X_train et y_train, cela peut être contrôlé en utilisant une option de la fonction. Voir la doc: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Note3: On obtient un mélange différent à chaque exécution du code. Le mélange peut être controllé si on désire pouvoir reproduire exactement la même expérience en fixant un "random_state". Voir doc: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.htm

### Évaluation du modèle

Évaluons maintenant les performances de notre modèle sur notre problème. Ce qui nous intéresse, c'est de savoir quelle(s) électrode(s) permet le mieux de faire la différence entre stimuli gauche ou droite. Intéressons nous d'abord par la première de la liste.

In [110]:
electrode = 0

Entraînons le modèle sur le dataset d'entraînement:

In [113]:
clf.fit(X_train[electrode], y_train)

KNeighborsClassifier()

Et utilisons le modèle entraîné pour prédire les classes du dataset de test:

In [116]:
y_pred = clf.predict(X_test[electrode])
print(y_pred)

[0 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1
 0 1 1 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0]


Pour mesurer les performances de notre algorithme nous pouvons simplement calculer son erreur et sa précision: pour ce faire, il suffit de calculer erreur = Somme(valeur_absolue(y_pred - y_test)) / nombre_de_predictions

puis accuracy = 1 - erreur

In [127]:
error = sum(abs(y_pred - y_test)) / len(y_pred)
accuracy = 1 - error
print(accuracy)

0.5277777777777778


Notre score est de 52.8% de précision (accuracy). Il existe d'autres mesures de performance du modèle, mais dans notre cas, la précision est adaptée.

Comme notre problème est un problème de classification binaire, le niveau de chance théorique est à 50%, cela veut dire que notre résultat est juste au dessus du niveau de chance et donc, que notre modèle est pas capable de faire la différence entre stimulus gauche et droite avec ces caractéristiques du signal là. 

Cependant, étant donné que nous avons un petit nombre de données, le niveau de chance théorique n'est pas le niveau de chance auquel on devrait comparer les performances de notre modèle. Il faut vérifier la pertinence statistique de ce résultat. Nous reviendrons sur cette partie plus tard.

Pour voir plus de métriques de performances, voir <a href="https://en.wikipedia.org/wiki/Fairness_(machine_learning)">ici</a>

Encore une fois, en pratique il est beaucoup plus simple d'utiliser directement une fonction donnée par sklearn pour calculer la précision de notre modèle:

In [129]:
score = clf.score(X_test[electrode], y_test)
print(score)

0.5277777777777778


Il ne nous reste donc plus qu'à répéter ces dernières opérations pour chaque électrode et de vérifier la pertinence statistique de nos résultats.

Seulement, le problème de la "Holdout method" est que si on l'utilise sur un petit jeu de données, il est possible que cette séparation nous avantage ou désavantage dans l'évalutation des performances de notre algorithme par chance uniquement. Pour remédier à celà, on utilise des méthodes de "cross-validation". Une cross-validation va répéter le processus de découpage des donées plusieurs fois afin de limiter l'impact du facteur chance au minimum.

### Cross-validation

Voilà quelques méthodes de cross-validation communes:
- Leave One Out (LOO)
- Leave P Out (LPO)
- K-Fold

LOO consiste à laisser de côté un seul exemple de notre jeu de données pour le test et de garder tout le reste pour l'entraînement. Bien que cette méthode permette de mesurer avec précision la performance d'un algorithme, elle est rarement utilisée en pratique. En effet, cette méthode est exhaustive, ce qui veut dire qu'elle va essayer toutes les combinaisons prossibles de train et de test sets. Ce n'est pas un problème dans notre cas vu que notre jeu de données est relativement petit (seulement 140 exemples), mais cela peut devenir un problème quand on possède des millions d'exemples et que effectuer l'entrainement et le test de l'algorithme prend un certain temps.

LPO est basée sur le même principe que LOO et est exhaustive. Cette méthode à l'avantage de tester sur plusieurs exemples au lieu d'un à chauque itération et est donc encore plus précise, mais elle souffre encore plus d'une augmentation de la taille des données, vu qu'il y a encore plus de combinaisons de p parmis n que de 1 parmis n.

K-fold consiste à couper les données en K sous-parties, de combiner K-1 sous parties pour constituer le train set et d'utiliser la dernière sous partie comme test set. Cette méthode est souvent préférée puisque beaucoup plus rapide que les précédentes quand on à accès à beaucoup de données tout en donnant une idée acceptable de la performance de l'algorithme.

Nous allons utiliser K-Fold puisque cette méthode est plus rapide tout en restant adaptée à notre problème.
En pratique on utilise la verison "stratifiée"de K-Fold afin de s'asurer que chacune des sous-parties du dataset contient bien autant d'exemples de chaque condition. Sans cela, il serait possible par malchance d'avoir quelques sous-parties des données qui ne contiennent qu'une conditon, ce qui fausserai les résultats.

On va choisir la valeur par défaut K = n_splits = 5, qui nous permettra d'avoir 5 scores de précision de notre algorithme entrainé sur K-1 = 4 / 5 = 80% de nos données et testé sur les 1/5 = 20% restantes.

In [ ]:
from sklearn.model_selection import StratifiedKFold as SKFold

cv = SKFold(n_splits=5)

Concrètement, ce que cette méthode va faire, c'est effectuer plusieures fois le train/test split que nous avons effectué plus haut manuellement.

De cette façon, au lieu d'obtenir un score unique de précision pour une électrode, on peut en obtenir plusieurs et les moyenner afin d'avoir un résultat plus robuste. Si on effectue une étape de cross-validation avec notre objet cv, on peut reproduire ce qu'on a fait plus haut:

In [ ]:
for train_index, test_index in cv.split(electrode, targets):
    X[electrode][train_index]

In [63]:

resultats = []
for electrode in data:
    predictions = []
    for train_index, test_index in cv.split(electrode, targets):
        clf = KNN()
        clf.fit(electrode[train_index], targets[train_index])
        pred = clf.predict(electrode[test_index])
        predictions.append(pred)
    resultats.append(predictions)


#### Explication du code:

Dans le code ci-dessus, on boucle à travers toutes les électrodes des données puis à travers toutes les instances de la cross-validation, en pseudo-code, ça donne:

- créer une liste qui contiendra les résultats pour chaque électrode.
- pour chaque électrode:
    - créer une liste vide qui contiendra les prédictions pour cette électrode
    - pour les K possibilitées de découpage des données:
        - remettre à zero le classifieur 
        - entraîner le classifieur sur les K-1 sous parties
        - tester le classifieur sur a sous-partie restante
        - enregistrer les prédictions dans une liste
    - enregistrer les prédictions de cette électrode dans la liste

### Évaluation des performances du modèle

Maintenant que le modèle a été testé pour chaque électrode avec la méthode de cross-validation choisie, explorons un peu les résultats

In [65]:
# On convertie la liste des résultats en numpy array, un format plus facile à manipuler:
resultats = np.array(resultats, dtype=object)

# On vérifie que nos résultats ont bien la bonne dimension:
print(resultats.shape)

(305, 5)


On obtiens une matrice de taille nombre_d_electrodes x nombres_de_folds. 

Affichons par exemple les performances de notre modèle sur la première électrode, sur le premier "fold" du K-Fold:

In [68]:
print(resultats[0][0])

[1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 1]


Ce qu'on voit ci-dessus ce sont les prédictions individuelles pour les n examples qui ont été mis de côté lors de l'évaluation des performances de notre modèle sur le premier fold de la cross-validation.